In [7]:
import numpy as np
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
URM_path="../data/interactions_and_impressions.csv"
URM_path2="../data/interactions_and_impressions_v3.csv"
URM_all_dataframe = pd.read_csv(URM_path)

URM_all_dataframe.columns = ["UserID", "ItemID", "others","Data"]
URM_all_dataframe=URM_all_dataframe.drop("others",axis=1)
URM_all_dataframe["Data"] = URM_all_dataframe["Data"].replace({0: 1, 1: 0})


URM_all_dataframe2 = pd.read_csv(URM_path2)
URM_all_dataframe2.columns = ["UserID", "ItemID","Data"]
#URM_all_dataframe2 = URM_all_dataframe2[URM_all_dataframe2['Data'] != 0.01]

URM_all_dataframe2 = URM_all_dataframe2.sort_values(by=['UserID', 'ItemID', 'Data'])
URM_all_dataframe = URM_all_dataframe.sort_values(by=['UserID', 'ItemID', 'Data'])

all_users_items=list(set(list(zip(URM_all_dataframe['UserID'],URM_all_dataframe['ItemID']))))
np.random.shuffle(all_users_items)


import random
train_percentage=0.8
user_for_training=random.sample(all_users_items, round(len(all_users_items)*train_percentage))
user_for_validation=list(set(all_users_items)-set(user_for_training))
user_for_training=pd.DataFrame(user_for_training,columns=["UserID","ItemID"])
user_for_validation=pd.DataFrame(user_for_validation,columns=["UserID","ItemID"])
data_train2 = URM_all_dataframe2[URM_all_dataframe2.set_index(["UserID","ItemID"]).index.isin(user_for_training.set_index(["UserID","ItemID"]).index)]
data_train = URM_all_dataframe[URM_all_dataframe.set_index(["UserID","ItemID"]).index.isin(user_for_training.set_index(["UserID","ItemID"]).index)]
#https://stackoverflow.com/questions/54006298/select-rows-of-a-dataframe-based-on-another-dataframe-in-python



data_valid2 = URM_all_dataframe2[URM_all_dataframe2.set_index(["UserID","ItemID"]).index.isin(user_for_validation.set_index(["UserID","ItemID"]).index)]
data_valid = URM_all_dataframe[URM_all_dataframe.set_index(["UserID","ItemID"]).index.isin(user_for_validation.set_index(["UserID","ItemID"]).index)]

import numpy as np
import scipy.sparse as sps


URM_train = sps.csr_matrix((data_train["Data"].values,
                          (data_train["UserID"].values, data_train["ItemID"].values)))
URM_train2 = sps.csr_matrix((data_train2["Data"].values,
                          (data_train2["UserID"].values, data_train2["ItemID"].values)))
URM_valid = sps.csr_matrix((data_valid["Data"].values,
                             (data_valid["UserID"].values, data_valid["ItemID"].values)))
URM_valid2 = sps.csr_matrix((data_valid2["Data"].values,
                             (data_valid2["UserID"].values, data_valid2["ItemID"].values)))
URM_train,URM_train2 ,URM_valid,URM_valid2

/var/folders/fv/ckspmkyx14g_h7sv41m5pjj80000gn/T/ipykernel_13220/1219749331.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  URM_all_dataframe = pd.read_csv(URM_path)


(<41629x24507 sparse matrix of type '<class 'numpy.int64'>'
 	with 1243712 stored elements in Compressed Sparse Row format>,
 <41629x24507 sparse matrix of type '<class 'numpy.float64'>'
 	with 1243704 stored elements in Compressed Sparse Row format>,
 <41629x24507 sparse matrix of type '<class 'numpy.int64'>'
 	with 310928 stored elements in Compressed Sparse Row format>,
 <41629x24507 sparse matrix of type '<class 'numpy.float64'>'
 	with 310928 stored elements in Compressed Sparse Row format>)

In [8]:
from Utils.Evaluator import EvaluatorHoldout

evaluator_validation=EvaluatorHoldout(URM_valid,cutoff_list=[10])
evaluator_validation2=EvaluatorHoldout(URM_valid2,cutoff_list=[10])


EvaluatorHoldout: Ignoring 1561 ( 3.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 302 ( 0.7%) Users that have less than 1 test interactions


In [9]:
#1
# BEST MODEL UNTIL NOW _ PROVA
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train2)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

Similarity column 24507 (100.0%), 5195.33 column/sec. Elapsed time 4.72 sec
Similarity column 41629 (100.0%), 2094.64 column/sec. Elapsed time 19.87 sec
[[-0.03821294  0.22342959  0.11251848 ... -0.24785024 -0.04001579
  -0.30580205]
 [-0.3757845  -0.39294732 -0.35206124 ... -0.39294732 -0.39294732
  -0.32898092]
 [ 0.46952245  1.1984674   0.87266195 ... -0.08048978 -0.28774083
  -0.06411874]
 ...
 [ 0.15822446  0.54429746  0.17842436 ... -0.10842475 -0.26598948
   0.05911335]
 [-0.19536361 -0.35831177 -0.37571847 ... -0.36288673 -0.38949785
  -0.3640929 ]
 [ 0.1742374   0.16388363 -0.37392926 ... -0.34736383 -0.3368826
  -0.2882854 ]]
[[-0.04810265 -0.00950444  0.03977475 ... -0.04346835 -0.00147495
  -0.03989626]
 [-0.05325403 -0.05325403 -0.05070062 ... -0.05325403 -0.05325403
  -0.05325403]
 [-0.04755964 -0.0030436   0.01227122 ... -0.03896819 -0.05325403
  -0.05325403]
 ...
 [-0.05115637 -0.04025374 -0.0477927  ... -0.04653176 -0.0519352
  -0.04750358]
 [-0.0484342  -0.05325403 -0

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),


[[ 0.3260757   1.271941    0.27305424 ... -0.3434443  -0.39825836
   0.10315936]
 [ 0.01721955  0.989554   -0.01973367 ... -0.18545571 -0.41348436
  -0.25207415]
 [-0.20648308 -0.40380737 -0.20433436 ... -0.48121616 -0.4517038
  -0.37384707]
 ...
 [-0.42901084 -0.33826905 -0.35193047 ... -0.41221166 -0.30024502
  -0.39417788]
 [ 1.3032042   2.2491713   1.3012707  ... -0.02233752 -0.12730716
   0.21915297]
 [-0.26406735  0.3347852  -0.13397771 ... -0.34739852 -0.4226748
  -0.45192683]]
[[-0.00256746 -0.04478959 -0.0472885  ... -0.05239188 -0.05239188
  -0.03985927]
 [-0.00478934  0.02367172 -0.02176101 ... -0.02557146 -0.0400175
  -0.03194363]
 [-0.04211553 -0.05026205 -0.05239188 ... -0.05239188 -0.05239188
  -0.05054987]
 ...
 [-0.05239188 -0.04745814 -0.04882948 ... -0.05239188 -0.04856559
  -0.05239188]
 [-0.03515085 -0.03011615 -0.02344912 ... -0.05239188 -0.04798562
  -0.05239188]
 [-0.05239188 -0.00775479 -0.04848404 ... -0.04219072 -0.04818689
  -0.05239188]]
[[-4.1039693e-01 -1

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.052458,0.092984,0.087597,0.026701,0.046143,0.173494,inf,0.06562,0.338724,0.213306,...,0.962502,0.326023,0.962502,0.028331,8.788871,0.991251,0.055781,0.651287,4.361689,0.26457


In [10]:
#1
# BEST MODEL UNTIL NOW - CONTROPROVA
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

Similarity column 24507 (100.0%), 5745.67 column/sec. Elapsed time 4.27 sec
Similarity column 41629 (100.0%), 2203.36 column/sec. Elapsed time 18.89 sec
[[-0.13684924 -0.21839945 -0.0609527  ... -0.2617832  -0.14067328
  -0.2578052 ]
 [-0.3005949  -0.3005949  -0.3005949  ... -0.3005949  -0.3005949
  -0.3005949 ]
 [ 0.3164673   0.21531178  0.5528693  ... -0.07181516 -0.24865656
  -0.24912004]
 ...
 [ 0.10544287 -0.05839789  0.22081172 ... -0.0177268  -0.19241671
  -0.11187489]
 [-0.12574479 -0.3005949  -0.3005949  ... -0.3005949  -0.3005949
  -0.3005949 ]
 [ 0.29078344  0.0480368  -0.3005949  ... -0.3005949  -0.23254912
  -0.3005949 ]]
[[-0.04417902 -0.02993349  0.02727542 ... -0.04417902  0.00873969
  -0.02289639]
 [-0.04417902 -0.04417902 -0.04417902 ... -0.04417902 -0.04417902
  -0.04417902]
 [-0.03804408 -0.04417902 -0.00483392 ... -0.03417346 -0.04417902
  -0.04417902]
 ...
 [-0.04417902 -0.04417902 -0.04417902 ... -0.03746466 -0.04417902
  -0.04417902]
 [-0.04417902 -0.04417902 -0

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),


[[ 0.470433    0.6623407   0.21907912 ... -0.29161444 -0.36791873
   0.1162551 ]
 [-0.02307301  0.4785657  -0.17808118 ... -0.15200752 -0.31484154
  -0.22309215]
 [-0.18423705 -0.3201931  -0.23636995 ... -0.3907925  -0.3907925
  -0.3907925 ]
 ...
 [-0.3605164  -0.3907925  -0.26477498 ... -0.2992098  -0.3907925
  -0.30276746]
 [ 0.99736196  1.2735873   0.9014764  ... -0.07619078 -0.10818016
  -0.16412719]
 [-0.17598864  0.16109881 -0.07150096 ... -0.24885586 -0.3907925
  -0.3907925 ]]
[[ 0.00141822 -0.042946   -0.04140949 ... -0.04766541 -0.04766541
  -0.03956923]
 [-0.0013858   0.05448312 -0.04766541 ... -0.02044889 -0.03009587
  -0.04766541]
 [-0.04299703 -0.04766541 -0.04766541 ... -0.04766541 -0.04766541
  -0.04766541]
 ...
 [-0.04766541 -0.04766541 -0.04766541 ... -0.04155254 -0.04766541
  -0.04766541]
 [-0.04256719 -0.04766541 -0.02529624 ... -0.04766541 -0.04231935
  -0.04766541]
 [-0.04766541 -0.00549826 -0.04766541 ... -0.03508546 -0.04766541
  -0.04766541]]
[[-0.39507806 -0.32

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.051707,0.090822,0.085454,0.026319,0.044859,0.170306,inf,0.064429,0.332984,0.209804,...,0.962502,0.320498,0.962502,0.039235,9.155334,0.992239,0.077249,0.678442,4.27269,0.264123


In [11]:
#1
# BEST MODEL UNTIL NOW - CONTROPROVA
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation2.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

Similarity column 24507 (100.0%), 6181.43 column/sec. Elapsed time 3.96 sec
Similarity column 41629 (100.0%), 2212.29 column/sec. Elapsed time 18.82 sec
[[-0.1324351  -0.2147122  -0.05586208 ... -0.2584826  -0.13629323
  -0.25446922]
 [-0.2976403  -0.2976403  -0.2976403  ... -0.2976403  -0.2976403
  -0.2976403 ]
 [ 0.32492197  0.22286484  0.56343114 ... -0.06682136 -0.24523897
  -0.2457066 ]
 ...
 [-0.20649374 -0.21364263 -0.20697181 ... -0.2976403  -0.2976403
   0.9116154 ]
 [ 0.4071608   0.5675374   1.196184   ... -0.19401403 -0.17066312
  -0.09014729]
 [-0.18384618  0.63878816 -0.02428077 ... -0.22977823 -0.01959523
  -0.22480163]]
[[-0.04388587 -0.02966148  0.02746253 ... -0.04388587  0.00895431
  -0.02263482]
 [-0.04388587 -0.04388587 -0.04388587 ... -0.04388587 -0.04388587
  -0.04388587]
 [-0.03776003 -0.04388587 -0.00459915 ... -0.03389515 -0.04388587
  -0.04388587]
 ...
 [-0.04388587 -0.04388587 -0.04388587 ... -0.04388587 -0.04388587
  -0.02550405]
 [-0.04388587  0.04418972  0

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


[[-0.2580474   0.17201282 -0.00148571 ... -0.20734037 -0.24288873
  -0.31511733]
 [-0.3676876  -0.07572895 -0.3251403  ... -0.3676876  -0.3676876
   0.18433769]
 [-0.127058   -0.28526843 -0.30567014 ... -0.3676876  -0.3676876
  -0.3676876 ]
 ...
 [-0.3676876  -0.3676876  -0.3086943  ... -0.3676876  -0.3676876
  -0.3676876 ]
 [ 0.06859826 -0.24724586 -0.10990764 ...  1.314375    0.22598839
   0.12236469]
 [-0.12597145 -0.01619515  0.12494736 ... -0.1618256  -0.1704599
  -0.31241548]]
[[-0.04675065 -0.03450454 -0.02651355 ... -0.03341787 -0.04675065
  -0.04675065]
 [-0.04675065 -0.04675065 -0.04675065 ... -0.04675065 -0.04675065
  -0.04675065]
 [-0.04675065 -0.03787962 -0.04675065 ... -0.04675065 -0.04675065
  -0.04675065]
 ...
 [-0.04675065 -0.04675065 -0.04675065 ... -0.04675065 -0.04675065
  -0.04675065]
 [-0.04675065 -0.04675065 -0.04675065 ... -0.04374719 -0.04411257
  -0.04675065]
 [-0.04675065 -0.02619126 -0.01616636 ... -0.02789673 -0.04675065
  -0.04675065]]
[[ 1.7274513   0.426

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.05553,0.075301,0.065149,0.028212,0.037591,0.178558,NaN,0.059956,0.349457,0.222157,...,0.992745,0.346922,0.992745,0.040207,9.191138,0.992349,0.079163,0.681095,4.245461,0.264775


In [12]:
#1
# BEST MODEL UNTIL NOW - CONTROPROVA
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train2)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation2.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

Similarity column 24507 (100.0%), 5248.65 column/sec. Elapsed time 4.67 sec
Similarity column 41629 (100.0%), 1549.37 column/sec. Elapsed time 26.87 sec
[[-0.03111871  0.23307617  0.12108312 ... -0.24280103 -0.03293914
  -0.3013182 ]
 [-0.3719833  -0.38931358 -0.34802863 ... -0.38931358 -0.38931358
  -0.32472318]
 [ 0.4815697   1.2176255   0.8886419  ... -0.07380797 -0.2830808
  -0.05727722]
 ...
 [-0.27160934 -0.2946201  -0.21650138 ... -0.36867473 -0.32366282
   0.63861746]
 [ 0.24877156  0.8237382   0.98096395 ... -0.2526713  -0.21996748
  -0.06061387]
 [-0.25450838  0.6468698  -0.07586192 ... -0.32443285 -0.12094796
  -0.2891493 ]]
[[-0.04776041 -0.00915187  0.04014052 ... -0.04312487 -0.00112022
  -0.03955183]
 [-0.05291317 -0.05291317 -0.05035908 ... -0.05291317 -0.05291317
  -0.05291317]
 [-0.04721726 -0.0026893   0.01262963 ... -0.03862351 -0.05291317
  -0.05291317]
 ...
 [-0.05291317 -0.05291317 -0.05291317 ... -0.05291317 -0.05291317
  -0.03463312]
 [-0.05291317  0.06837627 -

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


[[-0.1021753   1.209016    0.14641926 ... -0.16657017 -0.23103768
  -0.36366674]
 [ 0.622469   -0.27559415 -0.31132907 ... -0.45555967 -0.4149843
   0.05890389]
 [-0.27906418 -0.32693785 -0.40794605 ... -0.37269402 -0.45555967
  -0.41038394]
 ...
 [-0.45555967 -0.3380707  -0.40999892 ... -0.41007218 -0.45555967
  -0.45555967]
 [ 0.04450272 -0.07910332  0.12913568 ...  0.9138839  -0.14100881
   0.07153505]
 [-0.12176807  0.04809579  0.05616899 ... -0.27212116 -0.29039893
  -0.38127586]]
[[-0.0514411  -0.01345766 -0.02745397 ... -0.03419897 -0.04456975
  -0.0514411 ]
 [-0.03825311 -0.0514411  -0.04355268 ... -0.0514411  -0.04945733
  -0.0514411 ]
 [-0.0514411  -0.04073095 -0.0514411  ... -0.0514411  -0.0514411
  -0.04179064]
 ...
 [-0.0514411  -0.0514411  -0.0514411  ... -0.0514411  -0.0514411
  -0.0514411 ]
 [-0.0514411  -0.0514411  -0.04990627 ... -0.04883162 -0.04923519
  -0.0514411 ]
 [-0.04757444 -0.02542914 -0.02431217 ... -0.03193299 -0.0514411
  -0.0514411 ]]
[[ 1.5371445   0.318

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.057844,0.078901,0.068427,0.029377,0.039475,0.185829,NaN,0.062692,0.363685,0.231256,...,0.992745,0.361046,0.992745,0.0289,8.816294,0.99134,0.056901,0.653319,4.342845,0.265006
